# A general procedure for supervised learning

**Note: This is part 2 of CL1, you should complete part 1: "cl1_linear_regression.ipynb" before continuing with this one.**

For most of the supervised learning tasks, the procedure we follow is comprised of the following steps:

### Step 1: Data exploration
The first step is normally to load the data and try to understand its properties. A few things that are usually useful:
1. Check data formats.
2. Visual inspection of data.
3. Investigate (get some type of understanding for) how hard the problem is. 


### Step 2: Data preprocessing
1. Normalise (or scale) input data. 
2. Convert the data to a different type, or organize it differently for the optimization (e.g. Numpy arrays, subsets of the dataset, etc.)
3. Encode input and output data on a suitable form. For instance, we often use one-hot encoding to represent string variables.
4. Split data into training, validation and test sets.


### Step 3: Training
1. Build a tentative network architecture (could be the simplest one you think could work, or based in previous sucesses).
2. Select optimizer, performance measures and a few more hyperparameters. 
3. Train the network. 
4. Analyze performance on the training and validation sets. Adjust design decisions accordingly.


### Step 4: Assesment
1. Use the network for predictions in the test set.
2. Evaluate the final quality of the model. **Attention**: Once this is done, you shouldn't alter your model anymore, otherwise you need a new test set (if you want a good estimate of your model's generalization capacity).

We are going to apply most of these steps to the task of correctly classifying an Iris plant, given its morphologic features present in the IRIS dataset.

# 1. Data exploration

### 1.1  Import the necessary modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline

### 1.2 Read the dataset

In [ ]:
dataset = pd.read_csv("iris.csv")

In [ ]:
dataset.head()

### 1.3 Analyzing the data

For this task, we'll use all of the data, not focus on only one of the species or a subset of the features. The `plot` method can help us obtain different types of visualizations of the data in the `DataFrame`. For instance, we can use it to plot histograms of each feature.

In [ ]:
dataset.plot(kind='hist', bins=30, alpha=0.7, figsize=[15,6]);

This is somewhat informative, but we could get an even better grasp of the data by first separating it into the different species (it seems likely that different species will have different feature distributions), and then plotting the histograms.

However, if we let the `plot` method automatically create the histogram bins where it wants, each histogram might have different ranges, which would make it harder to compare them. Instead, we create the bins ourselves and pass that as an argument.

In [ ]:
# Remove the 'species' column, so we get only the numeric values of the dataset
features_dataset = dataset.drop('species', axis=1)

# Find maximum and minimum values
maxval = np.max(features_dataset.values)
minval = np.min(features_dataset.values)

# Create 30 linearly spaced numbers in this range
my_bins = np.linspace(minval, maxval, 30)
print(my_bins)

In [ ]:
# Get the names of the species
species_names = dataset['species'].unique()
print(species_names)

In [ ]:
# For each species name, plot a histogram
for name in species_names:
    dataset[dataset["species"]==name].plot(kind="hist", bins=my_bins, alpha=0.7, figsize=[15,4], title=name);

This confirms that different species do have substantial differences in the distributions of each feature, e.g. the Setosa species has shorter sepals than the others, etc. 

Another way to gain more insight about the data is using the method `pairplot`, from the seaborn python module. This shows scatter plots between all feature pairs (hence the time required to run it increases exponentially with the number of features!) and histograms for each feature, color-coded by the species.

In [ ]:
sns.pairplot(dataset, hue='species');

It's also helpful to check if the dataset is balanced. We can do so like this:

In [ ]:
# Fill in a dictionary with the number of ocurrences of each species
n = {}
for name in species_names:
    extract_rule = dataset['species']==name
    n[name] = len(dataset[extract_rule])
    
print(n)

This shows that each species occurs exactly 50 times in the dataset, so it's perfectly balanced.

# 2. Data preprocessing

### 2.1 Preparing input and output vectors

Now we need to prepare the data for the training. The first thing we should do is define the input and the output arrays for our network. 

Defining the input is as simple as extracting only the numeric columns of the dataset (this can also be conveniently done using the `drop` method, as done before).

In [ ]:
# Extract numerical values
x = dataset[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values

# Print first 10 rows
print(x[:10])

Creating the output vector requires one more step, because of the way we'll train our network. Since the optimizer needs to be able to compare the predictions made by the neural network (i.e. a numeric vector), with the desired output vector in order to decide how to alter the weights, it's usually necessary to encode the output vector in a numeric format, instead of strings:

In [ ]:
dataset['species'].values[:5]

First we create a handy function to map the species strings to numbers.

In [ ]:
def encode_species(species):
    if species == 'setosa':
        return 0
    if species == 'versicolor':
        return 1
    if species == 'virginica':
        return 2
    else:
        raise ValueError('Species \'{}\' is not recognized.'.format(species))

And them we use `map` to apply it to every element in the dataset.

In [ ]:
temp = map(encode_species, dataset['species'].values)
y = np.array(list(temp))

In [ ]:
y[0:5]

### 2.2 Test split

Secondly, in order to assess how well our classifier generalizes to new, unseen data, we would like to withhold part of the dataset from the training process. This withheld part is usually called the test set. 

Scikit-learn provides an easy way to do so, with the `train_test_split` method. 

In [ ]:
from sklearn.model_selection import train_test_split

This method randomly chooses which examples will be withheld, and here we want the test set to be comprised of approximately 30% of the samples.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10)

In [ ]:
x_train.shape

In [ ]:
x_train[:5]

In [ ]:
y_train.shape

In [ ]:
y_train[:5]

In [ ]:
x_test.shape

In [ ]:
y_test.shape

Now we can use `x_train` and `y_train` to train the network, and `x_test` and `y_test` to evaluate it.

### 2.3 Create the `TensorDataset` and the `DataLoader`

Now we make use of the `TensorDataset` and the `DataLoader` classes to make the training step simpler afterwards.

In [ ]:
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [ ]:
torch_x = torch.tensor(x_train, dtype=torch.float32)
torch_y = torch.tensor(y_train, dtype=torch.int64)

In [ ]:
t_dataset = TensorDataset(torch_x, torch_y)
t_data_loader = DataLoader(t_dataset, batch_size=32, shuffle=True)

Note: If we don't provide the argument `dtype`, the `tensor` function tries to infer the type of the `Tensor` that will be created from the type of the data supplied. In this case, this is a problem, since `x_train` is of type `float64`:

In [ ]:
x_train.dtype

But in deep learning usually memory is a bottleneck, so it's common to use `float32` for the parameters of the neural network. Having the input (in this case `x_train`) as `float64`, and the parameters of the net as `float32` causes PyTorch to throw an error, and is a common mistake.

We don't have the same problem with `y_train` because its type is already the correct one for the loss we will use.

# 3. Training

### 3.1 Setting up the optimization task

We will use a neural network which only has one hidden layer. The input to this hidden layer are the features for each sample, which are 4-dimensional. The output of it will be 3-dimensional, consisting of one number for each of the three classes, corresponding to how likely the model thinks the input is from that class.

In [ ]:
from torch import nn

In [ ]:
class LogisticRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(4, 3)

    def forward(self, xb):
        return self.lin(xb)
    
model = LogisticRegressor()

**Task**: to make sure you clearly understand what we're doing here, draw the network on a piece of paper.

Note we didn't use a softmax activation for this model. This is because the loss we will use, PyTorch's [`CrossEntropyLoss`](https://pytorch.org/docs/stable/nn.html#nllloss), already applies the (log of the) softmax function for us.

In [ ]:
loss_fn = nn.CrossEntropyLoss()

And we use Adam as the optimizer.

In [ ]:
from torch import optim
optimizer = optim.Adam(model.parameters(), lr=0.1)

### 3.2 Performing the optimization

Now that everything is set up, we can perform the optimization similarly to how we did it in the linear regression task:

In [ ]:
for epoch in range(20):
    losses = []
    for b_x, b_y in t_data_loader:
        pred = model(b_x)
        loss = loss_fn(pred, b_y)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    avg_loss = sum(losses)/len(losses)
    
    print('Epoch: {}\tAvg loss: {}'.format(epoch, avg_loss))

We can clearly see that the loss decreases as the optimization progresses. However, it's hard to know if a certain value of the loss, say 0.5, is good or bad, so we're not sure about the performance of the model. Because of this, it's usually informative to display other metrics of progress during training. 

### 3.3 Optimization + compute accuracy

One such metric that is easily interpretable is the accuracy of the model, defined as:

$ Acc = \frac{\# \text{Samples correctly classified}}{\# \text{Samples}} $

From the definition, we see that accuracy is always a value between 0 and 1. An accuracy of 0 means that every single prediction made by our model is wrong, and an accuracy of 1 the exact opposite. For real-world tasks, we usually obtain an accuracy between 0 and 1, and we aim to make it as high as possible. 

**Task**: what if our dataset was imbalanced? Would it still be a good idea to use accuracy?

In order to know if a certain sample was predicted correctly by the model, we'll use the highest predicted class as the choice to compare with ground truth. For instance, these are the input features and ground-truth for the first sample in our dataset:

In [ ]:
sample_x, sample_y = t_dataset[0]
print(sample_x)
print(sample_y)

And this is the prediction according to our trained model:

In [ ]:
model(sample_x)

We will choose the index of the highest prediction as the "hard" prediction of the model:

In [ ]:
model(sample_x).argmax()

And now we can compare the prediction with the ground truth, to know if the model was correct or not.

In [ ]:
model(sample_x).argmax() == sample_y

`1` here means `True`, and `0` False. We will do this for all samples during training for computing the accuracy. Additionally, the accuracy for each batch is not as informative as the accuracy in the entire dataset, so we will aggregate the number of correct predictions and display it once for each epoch:

In [ ]:
# Reset the model and the optimizer
model = LogisticRegressor()
optimizer = optim.Adam(model.parameters(), lr=0.1)


for epoch in range(20):
    losses = []
    n_correct = 0
    for b_x, b_y in t_data_loader:
        pred = model(b_x)
        loss = loss_fn(pred, b_y)
        losses.append(loss.item())
        
        # Compute number of correct predictions
        hard_preds = pred.argmax(dim=1)
        n_correct += (pred.argmax(dim=1) == b_y).sum().item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    accuracy = n_correct/len(t_dataset)
    avg_loss = sum(losses)/len(losses)
    
    print('Epoch: {}\tAvg loss: {:.3f} \tAccuracy: {:.2f}'.format(epoch, avg_loss, accuracy))

Now it's easier to interpret the model's performance (you should get a result close to 100% classification).

**Task:** Were you expecting overfitting? Why (not)?

### 3.4 Optimization + validation

If we don't train the model for enough steps, it won't reach a satisfactory performance. However, if we train it for too many steps it might overfit to the training data. One way to balance this tradeoff (and to tune hyper-parameters in general) is to split the training data in two sets.  One we will use to actually perform the backpropagation and update the weights (usually this one is then referred to as the training set), and another one which is not used for backpropagation, only for assessing the model's performance (usually referred to as the validation set). This way we can train the model in the training set and use the performance in validation set to determine if we're overfitting.

We can easily split our existing `t_dataset` using the `random_split` function from PyTorch. This function accepts as first argument the dataset we want to split and as the second argument a sequence of lengths for the new datasets.

In [ ]:
from torch.utils.data import random_split
train_t_dataset, val_t_dataset = random_split(t_dataset, [90, 15])

Now we create data loaders for each of the sets:

In [ ]:
train_t_data_loader = DataLoader(train_t_dataset, batch_size=32, shuffle=True)
val_t_data_loader = DataLoader(val_t_dataset, batch_size=105)

And perform the optimization, computing the train and validation loss and accuracies.

In [ ]:
# Reset the model and the optimizer
model = LogisticRegressor()
optimizer = optim.Adam(model.parameters(), lr=0.1)


for epoch in range(20):
    
    # Compute predictions and back-prop in the training set
    losses = []
    n_correct = 0
    for b_x, b_y in train_t_data_loader:
        pred = model(b_x)
        loss = loss_fn(pred, b_y)
        losses.append(loss.item())
        
        hard_preds = pred.argmax(dim=1)
        n_correct += (pred.argmax(dim=1) == b_y).sum().item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    train_accuracy = n_correct/len(train_t_dataset)
    train_avg_loss = sum(losses)/len(losses)    

        
    # Compute predictions in the validation set (with adagrad deactivated)
    losses = []
    n_correct = 0
    with torch.no_grad():
        for b_x, b_y in val_t_data_loader:
            pred = model(b_x)
            loss = loss_fn(pred, b_y)
            losses.append(loss.item())
            
            hard_preds = pred.argmax(dim=1)
            n_correct += (pred.argmax(dim=1) == b_y).sum().item()
        val_accuracy = n_correct/len(val_t_dataset)
        val_avg_loss = sum(losses)/len(losses)      
        
        
    display_str = 'Epoch {} '
    display_str += '\tLoss: {:.3f} '
    display_str += '\tLoss (val): {:.3f}'
    display_str += '\tAccuracy: {:.2f} '
    display_str += '\tAccuracy (val): {:.2f}'
    print(display_str.format(epoch, train_avg_loss, val_avg_loss, train_accuracy, val_accuracy))

Now we can clearly see if the model is performing well and whether or not it's overfitting to the training data.

# 4. Assesment

Finally, we would like to be able to evaluate how well the model can predict the class of new, unseen samples. This was the reason for withholding part of our data from the training process, so that now we have fresh, unseen samples. 

The idea now is to use the trained model to predict the class of each new sample, given its features, and then compare the predicted label with the correct label for each sample.

---

To compare the labels, we can use different techniques. As we saw before, we can compute the accuracy, but this time on the test set samples. However, although this helps us to evaluate the model's performance, it provides an incomplete picture. For instance, it doesn't explain the types of missclassifications we are doing.

So that we can gather more information about the quality of our classifier, we'll also compute the confusion matrix of its predictions. The confusion matrix is a table layout of the predictions of the classifier, in which each row represents the labels of the predicted class and each column the labels of the correct class.

---

To illustrate, imagine we train a classifier on samples that are either from the 'dog' class or the 'cat' class. After training, we show it 50 new samples. 30 of these new samples are cats, and 20 are dogs.

For the new cats, our classifier correctly predicts 28 of them, but in 2 samples it thinks they are from the 'dog' class. Further, the classifier correctly predicts 15 of the new dogs, and in 5 samples it thinks they are actually from the 'cat' class. 

The resulting confusion matrix for this example would be

<table>
  <tr>
    <th colspan="2" rowspan="2"></th>
      <th colspan="2"><b>Predicted label</b></th>
  </tr>
  <tr>
    <td>Cat</td>
    <td>Dog</td>
  </tr>
  <tr>
      <td rowspan="2"><b>True label</b></td>
    <td>Cat</td>
    <td>28</td>
    <td>2</td>
  </tr>
  <tr>
    <td>Dog</td>
    <td>5</td>
    <td>15</td>
  </tr>
</table>

Note that the element $C_{ij}$ ($i$-th row, $j$-th column), corresponds to the number of predictions of class $i$, when the true known class was the $j$-th class. This is not universal: some sources define the confusion matrix as the transpose of the one shown here. However, `sklearn` defines confusion matrices like this, so we'll adhere to this definition.

A handy way of computing the confusion matrix, given the predictions and the true labels, is to use the function [`confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) from the scikit-learn module.

The first step is to compute our predictions in the test set. 

In [ ]:
test_samples = torch.tensor(x_test, dtype=torch.float32)
test_labels = torch.tensor(y_test, dtype=torch.int64)

preds = model(test_samples).argmax(dim=1)

Now, to compute the accuracy, we'll do like we did during training, but for the test samples.

In [ ]:
acc = torch.sum(preds == test_labels).item()/len(preds)
print("Accuracy: %.2f" % acc)

And lastly, we can compute the confusion matrix using the `confusion_matrix` method from scikit-learn.

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels, preds)

**Task**: What can you conclude from this confusion matrix? Which classes are easy/hard to separate?